In [1]:
import os  
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'
alpha = 0.75

log_filepath  = './mobilenet_slim_n_0.75'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same',
                        depthwise_initializer="he_normal",depthwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(alpha*f2[0]), (1,1), strides=(1,1), padding='same',
               kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    return x

In [4]:
def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    # change stride
    x = Conv2D(int(alpha*32), (3,3), strides=(1,1), padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(1,),(128,)])# change stride
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(1,),(256,)])# change stride
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
            
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(1,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [5]:
img_input=Input(shape=(32,32,3))
output = MobileNet(img_input)
model=Model(img_input,output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 24)        672       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 24)        96        
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 24)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 24)        240       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 24)        96        
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 24)        0         
__________

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenet_slim_n_0.75.h5')

Epoch 1/300
782/782 [==============================] - 76s 97ms/step - loss: 2.3349 - acc: 0.3950 - val_loss: 2.1997 - val_acc: 0.4803
Epoch 2/300
782/782 [==============================] - 71s 90ms/step - loss: 1.9465 - acc: 0.5466 - val_loss: 1.8141 - val_acc: 0.6046
Epoch 3/300
782/782 [==============================] - 69s 88ms/step - loss: 1.7108 - acc: 0.6342 - val_loss: 1.6270 - val_acc: 0.6659
Epoch 4/300
782/782 [==============================] - 69s 88ms/step - loss: 1.5278 - acc: 0.6962 - val_loss: 1.5702 - val_acc: 0.6862
Epoch 5/300
782/782 [==============================] - 69s 88ms/step - loss: 1.3991 - acc: 0.7396 - val_loss: 1.4113 - val_acc: 0.7402
Epoch 6/300
782/782 [==============================] - 69s 88ms/step - loss: 1.2933 - acc: 0.7716 - val_loss: 1.3547 - val_acc: 0.7640
Epoch 7/300
782/782 [==============================] - 69s 88ms/step - loss: 1.2169 - acc: 0.7943 - val_loss: 1.3152 - val_acc: 0.7670
Epoch 8/300
782/782 [==============================] - 

782/782 [==============================] - 69s 88ms/step - loss: 0.2023 - acc: 0.9992 - val_loss: 0.5639 - val_acc: 0.9204
Epoch 122/300
782/782 [==============================] - 68s 87ms/step - loss: 0.2013 - acc: 0.9993 - val_loss: 0.5688 - val_acc: 0.9189
Epoch 123/300
782/782 [==============================] - 68s 87ms/step - loss: 0.2010 - acc: 0.9992 - val_loss: 0.5689 - val_acc: 0.9216
Epoch 124/300
782/782 [==============================] - 68s 87ms/step - loss: 0.2002 - acc: 0.9991 - val_loss: 0.5699 - val_acc: 0.9200
Epoch 125/300
782/782 [==============================] - 68s 87ms/step - loss: 0.1997 - acc: 0.9993 - val_loss: 0.5690 - val_acc: 0.9207
Epoch 126/300
782/782 [==============================] - 68s 87ms/step - loss: 0.1991 - acc: 0.9991 - val_loss: 0.5636 - val_acc: 0.9218
Epoch 127/300
782/782 [==============================] - 68s 87ms/step - loss: 0.1981 - acc: 0.9993 - val_loss: 0.5647 - val_acc: 0.9222
Epoch 128/300
782/782 [==============================] 

Epoch 181/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1659 - acc: 0.9998 - val_loss: 0.5615 - val_acc: 0.9226
Epoch 182/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1655 - acc: 0.9998 - val_loss: 0.5577 - val_acc: 0.9240
Epoch 183/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1649 - acc: 0.9998 - val_loss: 0.5559 - val_acc: 0.9245
Epoch 184/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1644 - acc: 0.9998 - val_loss: 0.5617 - val_acc: 0.9223
Epoch 185/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1639 - acc: 0.9997 - val_loss: 0.5646 - val_acc: 0.9221
Epoch 186/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1632 - acc: 0.9998 - val_loss: 0.5532 - val_acc: 0.9243
Epoch 187/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1627 - acc: 0.9998 - val_loss: 0.5544 - val_acc: 0.9237
Epoch 188/300
782/782 [==================

Epoch 241/300
782/782 [==============================] - 69s 88ms/step - loss: 0.1538 - acc: 0.9999 - val_loss: 0.5505 - val_acc: 0.9239
Epoch 242/300
782/782 [==============================] - 69s 89ms/step - loss: 0.1536 - acc: 1.0000 - val_loss: 0.5518 - val_acc: 0.9238
Epoch 243/300
782/782 [==============================] - 69s 89ms/step - loss: 0.1536 - acc: 1.0000 - val_loss: 0.5493 - val_acc: 0.9238
Epoch 244/300
782/782 [==============================] - 71s 91ms/step - loss: 0.1535 - acc: 0.9999 - val_loss: 0.5492 - val_acc: 0.9247
Epoch 245/300
782/782 [==============================] - 71s 91ms/step - loss: 0.1535 - acc: 0.9999 - val_loss: 0.5544 - val_acc: 0.9247
Epoch 246/300
782/782 [==============================] - 71s 91ms/step - loss: 0.1535 - acc: 0.9999 - val_loss: 0.5519 - val_acc: 0.9233
Epoch 247/300
782/782 [==============================] - 71s 91ms/step - loss: 0.1535 - acc: 0.9999 - val_loss: 0.5525 - val_acc: 0.9245
Epoch 248/300
782/782 [==================